In [1]:
from openai import OpenAI
client = OpenAI()
  
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4-turbo",
)

In [2]:
thread = client.beta.threads.create()

In [3]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [4]:
message

Message(id='msg_49oDg6nJj1tHC0T1nJUilrKW', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1713631539, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_i7ZeJHttx790f62MpEYisdCN')

In [5]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Certainly, Jane Doe! To solve the equation \(3x + 11 = 14\), let's first isolate the variable \(x\). We'll start by subtracting 11 from both sides of the equation. Let's do that now.
assistant > code_interpreter

from sympy import symbols, Eq, solve

x = symbols('x')
equation = Eq(3 * x + 11, 14)
solution = solve(equation, x)
solution

output >

[1]

assistant > After solving the equation \(3x + 11 = 14\), we find that \(x = 1\). If you have any more questions or need further assistance, feel free to ask, Jane Doe!

## Without streaming

In [6]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [7]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_TgJeFa99RhTwmYBmi50Rl1nD', assistant_id='asst_MnZkDAPQFYW72gyJivdPnmHU', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='After solving the equation \\(3x + 11 = 14\\), we find that \\(x = 1\\). If you have any more questions or need further assistance, feel free to ask, Jane Doe!'), type='text')], created_at=1713631724, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_a4FIJBV68wF6cceZ3r4081Ze', status=None, thread_id='thread_i7ZeJHttx790f62MpEYisdCN'), Message(id='msg_O2KXKTN9suoEnYdVbsUknoY5', assistant_id='asst_MnZkDAPQFYW72gyJivdPnmHU', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Certainly, Jane Doe! To solve the equation \\(3x + 11 = 14\\), let's first isolate the variable \\(x\\). We'll start by subtracting 11 from both sides of the equation. Let's do that now."), type='text')], created_at=1713631